In [16]:
import requests
from pyquery import PyQuery as pq


In [105]:
URL = "https://results.chicagomarathon.com/2021/?page={page}&event=MAR&lang=EN_CAP&num_results=1000&pid=list&search%5Bsex%5D=M&search%5Bage_class%5D=%25"

In [107]:
def parse_page(URL):
    resp = requests.get(URL)
    d = pq(resp.content)
    # find first name field and navigate up to overarching row
    all_runners = d(".list-field.type-fullname a").closest(".list-group-item .row")
    all_runners_parsed = []
    for runner in all_runners.items():
    #     print(runner)
        name_country = runner.find(".type-fullname a").text()
        data = {
            "name": name_country[:-6],
            "country": name_country[-4:-1]
        }
        all_runners_parsed.append(data)

    return all_runners_parsed

In [112]:
all_runners = []

for page in range(1, 16):
    print("Parsing page: " + str(page))
    all_runners += parse_page(URL.format(page=1))

print(all_runners)

Parsing page: 14
Finished page: 14
Parsing page: 15
Finished page: 15
[{'name': 'Tura Abdiwak, Seifu', 'country': 'ETH'}, {'name': 'Rupp, Galen', 'country': 'USA'}, {'name': 'Kiptanui, Eric', 'country': 'KEN'}, {'name': 'Suzuki, Kengo', 'country': 'JPN'}, {'name': 'Tamru Aredo, Shifera', 'country': 'ETH'}, {'name': 'Mickow, Colin', 'country': 'USA'}, {'name': 'Montanez, Nico', 'country': 'USA'}, {'name': 'Kipyego, Reuben Kiprop', 'country': 'KEN'}, {'name': 'Fischer, Reed', 'country': 'USA'}, {'name': 'Given, Wilkerson', 'country': 'USA'}, {'name': 'Panning, Zachery', 'country': 'USA'}, {'name': 'Thomson, Jacob', 'country': 'USA'}, {'name': 'Young, Clayton', 'country': 'USA'}, {'name': 'Wiley, Turner', 'country': 'USA'}, {'name': 'Derrick, Chris', 'country': 'USA'}, {'name': 'Izewski, Josh', 'country': 'USA'}, {'name': 'Butler, Ian', 'country': 'USA'}, {'name': 'Phillips, Jonathan', 'country': 'USA'}, {'name': 'Havel, Kevin', 'country': 'USA'}, {'name': 'Sayenko, Mike', 'country': 'USA

In [113]:
len(all_runners)

2000

In [89]:
l

[<li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list-active.list-group-item.row>, <li..list-group-item.row>, <li..list

In [75]:
print(l.closest(".list-group-item .row").eq(1))

<li class=" list-group-item row">
<div class="col-xs-12 col-sm-12 col-md-5 list-field-wrap">
  <div class="row">
  <div class=" list-field type-place place-secondary hidden-xs numeric" style="width: 40px">2</div>
  <div class=" list-field type-place place-primary numeric" style="width: 40px">2</div>
  <h4 class=" list-field type-fullname"><a href="?content=detail&amp;fpid=list&amp;pid=list&amp;idp=LSMG963824DD61&amp;lang=EN_CAP&amp;event=MAR&amp;lang=EN_CAP&amp;num_results=1000&amp;search%5Bsex%5D=M&amp;search%5Bage_class%5D=%25&amp;search_event=MAR">Rupp, Galen (USA)</a></h4>
  </div>
</div>
<div class="col-xs-12 col-sm-12 col-md-7 list-field-wrap">
<div class="pull-left">
  <div class="row">
  <div class=" list-field type-field" style="width: 45px"><div class="visible-xs-block visible-sm-block list-label">BIB</div>9</div>
  <div class=" list-field type-age_class" style="width: 90px"><div class="visible-xs-block visible-sm-block list-label">Division</div>35-39</div>
  </div>
</div>
<d